In [21]:
import os
from glob import glob

import xarray as xr
from regridding import regrid

import gev_stat_utils as gevsu

from utils import gev_metric_ids
from utils import roar_data_path as project_data_path

In [2]:
from dask.distributed import LocalCluster
cluster = LocalCluster(n_workers=10, threads_per_worker=1)
client = cluster.get_client()

In [3]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 10
Total threads: 10,Total memory: 40.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38113,Workers: 10
Dashboard: /proxy/8787/status,Total threads: 10
Started: Just now,Total memory: 40.00 GiB
Comm: tcp://127.0.0.1:41007,Total threads: 1
Dashboard: /proxy/45765/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:41457,


In [4]:
import gev_nonstat_utils as gevns

In [5]:
import importlib
importlib.reload(gevns)

<module 'gev_nonstat_utils' from '/storage/work/dcl5300/current_projects/conus_comparison_lafferty-etal-2024/src/gev_nonstat_utils.py'>

In [22]:
params_in = xr.open_dataset('/storage/group/pches/default/users/dcl5300/conus_comparison_lafferty-etal-2024/extreme_value/original_grid/max_tasmax/LOCA2_ACCESS-CM2_r1i1p1f1_ssp370_1950-2100_nonstat_mle_main.nc')

In [23]:
params_in

<xarray.Dataset> Size: 115MB
Dimensions:             (gcm: 1, member: 1, ssp: 1, ensemble: 1, quantile: 1,
                         lat: 474, lon: 944, time: 7)
Coordinates:
  * gcm                 (gcm) <U10 40B 'ACCESS-CM2'
  * member              (member) <U8 32B 'r1i1p1f1'
  * ssp                 (ssp) <U6 24B 'ssp370'
  * ensemble            (ensemble) <U5 20B 'LOCA2'
  * quantile            (quantile) <U4 16B 'main'
  * lat                 (lat) float32 2kB 23.91 23.97 24.03 ... 53.41 53.47
  * lon                 (lon) float32 4kB 234.5 234.6 234.7 ... 293.4 293.5
  * time                (time) int64 56B 1950 1975 2000 2025 2050 2075 2100
Data variables:
    loc_intcp           (gcm, member, ssp, ensemble, quantile, lat, lon) float64 4MB ...
    loc_trend           (gcm, member, ssp, ensemble, quantile, lat, lon) float64 4MB ...
    scale               (gcm, member, ssp, ensemble, quantile, lat, lon) float64 4MB ...
    shape               (gcm, member, ssp, ensemble, quantile, lat, lon) float64 4MB ...
    10yr_return_level   (gcm, member, ssp, ensemble, quantile, time, lat, lon) float64 25MB ...
    25yr_return_level   (gcm, member, ssp, ensemble, quantile, time, lat, lon) float64 25MB ...
    50yr_return_level   (gcm, member, ssp, ensemble, quantile, time, lat, lon) float64 25MB ...
    100yr_return_level  (gcm, member, ssp, ensemble, quantile, time, lat, lon) float64 25MB ...

In [8]:
%%time
out = gevns.fit_ns_gev_xr_bootstrap(
    params_in = params_in,
    metric_id = 'max_tasmax',
    expected_length = 151,
    starting_year = 1950,
)

/storage/work/dcl5300/current_projects/conus_comparison_lafferty-etal-2024/.venv/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,


CPU times: user 1min 1s, sys: 6.21 s, total: 1min 7s
Wall time: 7min 32s


In [26]:
8/(100*100)*(474*944)/60

5.966080000000001

In [25]:
34/(200*200)*(474*944)/60

6.338959999999999

In [16]:
48*11/4/24

5.5

In [4]:
200*200/(10*10)

400.0

In [19]:
import numpy as np
from joblib import Parallel, delayed
from functools import partial

# Assuming these are defined elsewhere
# _gev_fit_parametric_bootstrap_1d_nonstationary, expected_length, n_boot, fit_method, params_in

def process_chunk(chunk, fit_function):
    """Process a single data chunk by applying the fit function"""
    return np.apply_along_axis(fit_function, axis=0, arr=chunk)

# Prepare data
data = np.squeeze(
    params_in.isel(lat=slice(100,200), lon=slice(200,300))[["loc_intcp", "loc_trend", "scale", "shape"]]
    .to_array()
    .to_numpy()
)

# Create chunks similar to your Dask implementation
chunks = []
chunk_size = (10, 10)  # Assuming your original chunks were (4, 10, 10)
for i in range(0, data.shape[1], chunk_size[0]):
    for j in range(0, data.shape[2], chunk_size[1]):
        chunk = data[:, i:i+chunk_size[0], j:j+chunk_size[1]]
        if chunk.size > 0:  # Ensure we don't add empty chunks
            chunks.append(chunk)

# Prepare fit function
fit_gev_1d = partial(
    gevns._gev_fit_parametric_bootstrap_1d_nonstationary,
    expected_length=151,
    n_boot=100,
    fit_method='mle',
)

In [20]:
%%time
# Run parallel processing
results = Parallel(n_jobs=-1, verbose=10)(
    delayed(process_chunk)(chunk, fit_gev_1d) for chunk in chunks
)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   33.8s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done  41 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done  65 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done  78 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done  92 out of 100 | elapsed:  5.8min remaining:   30.4s


CPU times: user 146 ms, sys: 172 ms, total: 318 ms
Wall time: 5min 56s


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  5.9min finished


In [33]:
len(results)

100

In [30]:
results[2].shape

(100, 4, 10, 10)

In [32]:
results[3][0, :, :, :]

array([[[4.16637737e+01, 4.17835294e+01, 4.12324254e+01, 4.04198425e+01,
         3.94908824e+01, 3.85970591e+01, 3.93132966e+01, 3.95582706e+01,
         3.86395902e+01, 3.57994259e+01],
        [4.05058387e+01, 3.95208520e+01, 4.05527569e+01, 3.98149435e+01,
         3.89931309e+01, 3.87982169e+01, 3.94205612e+01, 3.97851134e+01,
         3.85858792e+01, 3.76727794e+01],
        [4.00420069e+01, 3.98723439e+01, 3.96571001e+01, 3.98529589e+01,
         3.87790626e+01, 3.83451711e+01, 4.06084272e+01, 4.01176540e+01,
         3.86655918e+01, 3.86521818e+01],
        [4.05522258e+01, 4.04860995e+01, 3.96721192e+01, 3.85933134e+01,
         3.79898366e+01, 3.84832470e+01, 4.08598229e+01, 4.02655483e+01,
         3.89537432e+01, 3.94279346e+01],
        [4.04554255e+01, 4.04993803e+01, 3.98741512e+01, 3.89085857e+01,
         3.78182777e+01, 3.87124429e+01, 4.00182147e+01, 3.95654715e+01,
         3.87548531e+01, 3.88182694e+01],
        [4.05160964e+01, 3.98255026e+01, 3.99326038e+01, 3.9

In [22]:
lat_name = 'lat'
lon_name = 'lon'

out_test = xr.Dataset(
        data_vars={
            "loc_intcp": (["n_boot", lat_name, lon_name], results[:, 0, :, :]),
            "loc_trend": (["n_boot", lat_name, lon_name], results[:, 1, :, :]),
            "scale": (["n_boot", lat_name, lon_name], results[:, 2, :, :]),
            "shape": (["n_boot", lat_name, lon_name], results[:, 3, :, :]),
        },
        coords={
            "n_boot": (["n_boot"], np.arange(100)),
            lat_name: ([lat_name], params_in.isel(lat=slice(100,200), lon=slice(200,300))[lat_name].to_numpy()),
            lon_name: ([lon_name], params_in.isel(lat=slice(100,200), lon=slice(200,300))[lon_name].to_numpy()),
        },
    )

TypeError: list indices must be integers or slices, not tuple

In [ ]:

# Reconstruct the output array
# This will depend on your specific requirements
# Here's a simple example that concatenates along axis 1
out = np.concatenate(results, axis=1)  # Adjust axis as needed

In [2]:
############
### Dask ###
############
from dask_jobqueue import SLURMCluster

cluster = SLURMCluster(
    account="pches_cr_default",
    queue='basic',
    cores=1,
    processes=1,
    memory="80GiB",
    walltime="02:00:00",
)

cluster.scale(jobs=24)  # ask for jobs

from dask.distributed import Client
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.6.10.89:33771,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


## Fit GEV

In [4]:
# Stationary, L-moments, main
for metric_id in gev_metric_ids:
    gevsu.gev_fit_all(
        metric_id=metric_id,
        stationary=True,
        fit_method="lmom",
        periods_for_level=[10, 25, 50, 100],
        levels_for_period=None,
        proj_years=[2050, 2100],
        hist_years=[1950, 2014],
        bootstrap=False,
    )

CPU times: user 13.8 s, sys: 829 ms, total: 14.7 s
Wall time: 31.7 s


In [3]:
## Stationary, L-moments, bootstrap

# Can separate STAR-ESDM since it's higher resolution
# and requires significantly more memory

# 40GB is enough for LOCA & GARD-LENS
# 80GB needed for STAR
for metric_id in gev_metric_ids:
    gevsu.gev_fit_all(
        metric_id=metric_id,
        stationary=True,
        fit_method="lmom",
        periods_for_level=[10, 25, 50, 100],
        levels_for_period=None,
        proj_years=[2050, 2100],
        hist_years=[1950, 2014],
        bootstrap=True,
        n_boot_proj=100,
        n_boot_hist=1,
        include_STAR_ESDM=True,
    )

In [4]:
client.shutdown()

## Regridding

In [5]:
%%time
# Regrid to LOCA using NN
method = "nearest"
target = "LOCA2"

for metric_id in gev_metric_ids:
    # Save path
    store_path = f"{project_data_path}/extreme_value/loca_grid/{metric_id}"
    # Get all files
    files = glob(
        f"{project_data_path}/extreme_value/original_grid/{metric_id}/STAR-ESDM*"
    ) + glob(f"{project_data_path}/extreme_value/original_grid/{metric_id}/GARD-LENS*")

    # Loop through
    for file in files:
        # Check if done
        file_name = file.split("/")[-1]
        store_name = file_name.replace(".nc", f"_{method}.nc")
        if not os.path.exists(f"{store_path}/{store_name}"):
            # Regrid and store
            ds_tmp = xr.open_dataset(file)
            ds_out = regrid(ds_tmp, method=method, target=target)
            ds_out.to_netcdf(f"{store_path}/{store_name}")

CPU times: user 2min 40s, sys: 3min 7s, total: 5min 48s
Wall time: 7min 47s


In [6]:
%%time
# Regrid to GARD-LENS using NN
method = "nearest"
target = "GARD-LENS"

for metric_id in gev_metric_ids:
    # Save path
    store_path = f"{project_data_path}/extreme_value/gard_grid/{metric_id}"
    # Get all files
    files = glob(
        f"{project_data_path}/extreme_value/original_grid/{metric_id}/STAR-ESDM*"
    ) + glob(f"{project_data_path}/extreme_value/original_grid/{metric_id}/LOCA2*")

    # Loop through
    for file in files:
        # Check if done
        file_name = file.split("/")[-1]
        store_name = file_name.replace(".nc", f"_{method}.nc")
        if not os.path.exists(f"{store_path}/{store_name}"):
            # Regrid and store
            ds_tmp = xr.open_dataset(file)
            ds_out = regrid(ds_tmp, method=method, target=target)
            ds_out.to_netcdf(f"{store_path}/{store_name}")

CPU times: user 1min 23s, sys: 1min 45s, total: 3min 8s
Wall time: 4min 51s
